# Utils.py

First, let's take a look at utils.py, there are many functions which are useful.
For example, the gaussian kernel implementation and the matrix kernel basic calculus:

In [1]:
def gaussian_kernel(gamma, x_1, x_2):
    """
    Compute the gaussian kernel between two vectors.

    :param gamma: float, gamma value
    :param x_1: array
    :param x_2: array
    :returns: float, the kernel value
    """
    return np.exp((-1) * gamma * np.linalg.norm(x_1 - x_2)**2)

In [2]:
def get_kernel_matrix(dataframe, gamma):
    """
    Take dataframe and compute the kernel matrix associated.
    :param dataframe: a df which size is (n x d)
    :param gamma: float, parameter for the shift of the gaussian kernel
    :return: matrix, the kernel matrix (of size n x n)
    """
    n = len(dataframe)
    kernel_matrix = np.zeros((n, n))  # Initialize the kernel matrix
    for i in range(n):
        for j in range(0, i):
            kernel_matrix[i, j] = gaussian_kernel(gamma, dataframe.iloc[i], dataframe.iloc[j])

    # Then we do a transposition because the kernel matrix is symetric
    return kernel_matrix + kernel_matrix.transpose()

Then here is the function for the low-rank pseudo-inverse approximation:

In [3]:
def low_rank_approx(matrix, rank):
    """
    Compute the pseudo-inverse of a low k-rank approximation of a matrix.
    :param matrix: matrix to get the approximation
    :param rank: int, the rank of the matrix approximation
    :returns: a matrix which is the pseudo-inverse of the approximation
    """

    u, s, v = np.linalg.svd(matrix, full_matrices=False)

    M_k = np.zeros((len(u), len(v)))
    for i in range(rank):
        M_k += s[i] * np.outer(u.T[i], v[i])
    pseudo_inverse_M_k = np.linalg.pinv(M_k)
    return pseudo_inverse_M_k


And the function to get the decomposition for the W vector:

In [4]:
def get_decomposition(sample, df, gamma):
    """
    Compute the decomposition of the matrix W.
    :param sample: matrix of size m x d
    :param df: matrix of size n x d
    :param gamma: float, parameter for the gaussian shift
    :return: The W matrix of size n x m
    """
    number_sample_m = len(sample)
    W = np.zeros((len(df), number_sample_m))  # Create the matrix
    for j in range(number_sample_m):
        for i in range(len(df)):
            W[i, j] = gaussian_kernel(gamma, sample.iloc[j], df.iloc[i])

    return W

Finally we can perform the Nystrom approximation everything matrix, we have to indicate the rank and the size of the sample:

In [5]:
def nystorm_approximation(df, number_sample_m, rank_k, gamma):
    """
    Compute the Nystorm approximation of the dataframe.

    :param df: The dataframe which has the data points.
    :param number_sample_m: The size of the sample to take.
    :param rank_k: The rank of the matrix for the approximation.
    :param gamma: float, the parameter for the shift
    :return: W (matrix of size n x m) and L (matrix of size m x m)
    """

    sample = df.sample(n=number_sample_m, random_state=0, axis=0)

    kernel_matrix = get_kernel_matrix(sample, gamma)  # Compute the kernel matrix
    M_k = low_rank_approx(kernel_matrix, rank_k)  # Get the pseudo-inverse of the low rank approximation
    L = M_k
    W = get_decomposition(sample, df, gamma)  # Get the vector of the decomposition

    return W, L

# meka.py

This file is composed of the class of the meka algorithm, we initiate it with the following parameters. We are going to store each block of W and L in an array and a dictionnary with corresponding indexes:

In [6]:
    def __init__(self, df, gamma, rank_k, number_clusters_c, number_sample_m):
        """
        Initialize the algorithm.
        :param df: Dataframe with the values
        :param gamma: float, parameter for the gaussian shift.
        :param rank_k: int, rank of the low-rank approximation matrix.
        :param number_clusters_c: int, number of blocks (clusters)
        :param number_sample_m: int, size of the sample for the decomposition.
        :return: G_hat, matrix based on the MEKA algorith
        """
        self.df = df
        self.gamma = gamma
        self.rank_k = rank_k
        self.number_clusters_c = number_clusters_c
        self.number_sample_m = number_sample_m

        self.blocks_of_L = {}
        self.blocks_of_W = []

We have already performed the K-means partitioning, then we extract the dataframe corresponding to each cluster and we do a Nystorm approximation of it based on the function in utils.py.

In [7]:
    def compute_on_diagonal_blocks(self):
        """
        Compute the on-diagonal blocks by using the nystorm approximation for each block.
        :return:
        """
        for cluster in range(self.number_clusters_c):
            df_cluster = self.df.loc[self.df['clusters'] == cluster]  # Extract the cluster
            df_cluster = df_cluster.iloc[:, :-1]  # Delete the last column

            W, L = nystorm_approximation(df_cluster, self.number_sample_m, self.rank_k, self.gamma)

            # We can get the block G(s,s) by doing:
            # G = W * M_k * transpose(W)

            self.blocks_of_W.append(W)
            position = str(cluster) + '_' + str(cluster)
            self.blocks_of_L[position] = L  # Store the L(s,s) element

            print("Success for the cluster " + str(cluster))

Then we can compute the second step, we randomly extract a submatrix \hat{G} and we get the value of the vector L  based on the least square solution of the equation:

In [8]:
    def compute_off_diagonal_blocks(self):
        """
        Compute the off-diagonal blocks. We use the blocks of W calculated previously.
        :return:
        """

        dim_subsample = 3 * self.rank_k  # Get the size of the subsample matrix
        for s in range(self.number_clusters_c):
            for t in range(self.number_clusters_c):
                if s != t:

                    df_s = self.df.loc[self.df['clusters'] == s].iloc[:, :-1]
                    df_t = self.df.loc[self.df['clusters'] == t].iloc[:, :-1]

                    number_col_s = df_s.shape[0]
                    number_row_t = df_t.shape[0]

                    index_col_s = random.randint(0, number_col_s - dim_subsample)
                    index_row_t = random.randint(0, number_row_t - dim_subsample)

                    # We have the coordinates of the submatrix

                    sub_matrix = np.zeros((dim_subsample, dim_subsample))

                    for i in range(index_row_t, index_row_t + dim_subsample):
                        for j in range(index_col_s, index_col_s + dim_subsample):
                            sub_matrix[i - index_row_t, j - index_col_s] = gaussian_kernel(self.gamma, df_s.iloc[j],
                                                                                           df_t.iloc[i])

                    # Then we can extract W_nu_s and W_nu_t

                    W_s = self.blocks_of_W[s]
                    W_nu_s = W_s[index_col_s:index_col_s + dim_subsample, :]

                    W_t = self.blocks_of_W[t]
                    W_nu_t = W_t[index_row_t:index_row_t + dim_subsample, :]

                    # And finally we compute the dot product (i.e solve Least Squares) to get L(s,t)

                    L_s_t = inv(np.transpose(W_nu_s).dot(W_nu_s)).dot(np.transpose(W_nu_s)).dot(sub_matrix).dot(
                        W_nu_t).dot(
                        inv(np.transpose(W_nu_t).dot(W_nu_t)))

                    self.blocks_of_L[str(s) + '_' + str(t)] = L_s_t

                    # We have now all the blocks of L and W

We have now calculated all the blocks of L and W. By doing a block product of matrices, we can reconstruct G hat,
we have to be careful about the indexes value.

In [9]:
    def reconstruct_G_hat(self):
        """
        Reconstruct G hat based on the blocks decomposition.
        :return: the matrix G_hat of size (n x n)
        """
        n = len(self.df)
        G_hat = np.zeros((n, n))

        index_row = 0
        for i in range(self.number_clusters_c):
            W_1 = self.blocks_of_W[i]
            index_col = 0
            for j in range(self.number_clusters_c):
                W_2 = np.transpose(self.blocks_of_W[j])
                L = self.blocks_of_L[str(i) + '_' + str(j)]
                G_partial = W_1.dot(L).dot(W_2)
                a, b = G_partial.shape
                G_hat[index_row:index_row + a, index_col:index_col + b] = G_partial

                index_col += b
            index_row += a

        print('Construction of G_hat is finished.')
        return G_hat

# main.py

Here is the main file. We get data from ijcnn1, we randomly sample 20 000 data points from it then we create the real G (kernel matrix), we compute a G_nystorm matrix thanks to the algorithm in utils.py.

In [ ]:
df = pd.read_csv('data/ijcnn1.csv', sep=',').iloc[:, 1:]
df = df.sample(n=20000, random_state=0)
rank_k = 128
number_sample_m = 300
gamma = 1

G = get_kernel_matrix(df, gamma=gamma)


W, L = nystorm_approximation(df, number_sample_m, rank_k, gamma)
G_nystrom = W.dot(L).dot(np.transpose(W))

Finally we do a k-means clustering and perform the MEKA algorithm to get the third matrix G_hat based on the MEKA algorithm:

In [ ]:
number_clusters_c = 10
# Do the Kmeans clustering and group data points based on that
kmeans = KMeans(n_clusters=number_clusters_c, random_state=0).fit(df)

df['clusters'] = kmeans.labels_
df_c = df.sort_values(by='clusters')
df = df.drop(['clusters'], axis=1)

meka_algorithm = Meka(df_c, gamma, rank_k, number_clusters_c, number_sample_m=20)
meka_algorithm.compute_on_diagonal_blocks()
meka_algorithm.compute_off_diagonal_blocks()

G_hat = meka_algorithm.reconstruct_G_hat()

We get the following score which evaluate the approximation of the kernel matrix based on the Froebenius norm:

In [ ]:
score = np.linalg.norm(G - G_nystrom, 'fro') / np.linalg.norm(G, 'fro')

score_2 = np.linalg.norm(G - G_hat, 'fro') / np.linalg.norm(G, 'fro')

Unfortunately my computer was not able to compute the kernel matrix with 20 000 sample data points but when I take 2000 data points and adapt the parameters accordingly I'm able to obtain a score of 0.29 for the normal Nystrom method and around 0.18 for the MEKA algorithm.


BOUTHEMY Marin, ENSAE 3A